# Loan 데이터 분류나무

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('loan.csv')

In [ ]:
df.isnull().sum().sum()

In [ ]:
df['job'] = df['job'].replace({'Office':0, 'ProfExe':1, 'Other':2, 'Mgr':3, 'Self':4, 'Sales':5})
df.groupby('job').mean()

### 나무 모형

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
X = df.drop(['y'], axis=1)
y = df['y']
xname = X.columns
yname = ['Normal','Bad']

### 모든 Alpha 와 Impurity의 계산

In [ ]:
c_tree = DecisionTreeClassifier(random_state=0) 
path = c_tree.cost_complexity_pruning_path(X, y)
path = pd.DataFrame(path)
path

### Alpha 와 Impurity의 관계

In [ ]:
fig, ax = plt.subplots()
ax.plot(path.ccp_alphas, path.impurities, marker='o', drawstyle="steps-post")
ax.set_xlim(0.1, 0)  # decreasing order
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.plot(path.ccp_alphas, path.impurities, marker='o', drawstyle="steps-post")
ax.set_xlim(0.004, 0)  # decreasing order
plt.show()

### 최적의 Alpha를 찾는 방법

In [ ]:
from sklearn.model_selection import GridSearchCV
np.random.seed(0)
g_cv = GridSearchCV(DecisionTreeClassifier(random_state=0),
              param_grid={'ccp_alpha': np.arange(0.000, 0.005, 0.0001)}, cv=10, n_jobs=-1)
g_cv.fit(X, y)

In [ ]:
print('selected:',g_cv.best_params_)
print('score   :',g_cv.best_score_)

### Alpha=0.0033 트리

In [ ]:
c_tree = DecisionTreeClassifier(ccp_alpha=0.0033,random_state=0) 
c_tree.fit(X, y)
plt.figure(figsize=(15,10))
plot_tree(c_tree, feature_names=xname, class_names=yname, filled=True, fontsize=10) 
plt.show()

### 하이퍼파라미터 튜닝

In [ ]:
np.random.seed(0)
g_cv = GridSearchCV(DecisionTreeClassifier(random_state=0),
              param_grid={'ccp_alpha': np.arange(0.000, 0.005, 0.0001),
                          'min_impurity_decrease': np.arange(0,0.003,0.0005),
                          'min_samples_split': np.arange(800,1200,200)},
                    cv=10, n_jobs=-1)
g_cv.fit(X, y)

In [ ]:
print('selected:',g_cv.best_params_)
print('score   :',g_cv.best_score_)

### Alpha=0.0, min_impurity_decrease=0.001,  min_samples_split=1000  트리

In [ ]:
c_tree = DecisionTreeClassifier(ccp_alpha=0.0,min_impurity_decrease=0.001,min_samples_split=1000,random_state=0) 
c_tree.fit(X, y)
plt.figure(figsize=(15,15))
plot_tree(c_tree, feature_names=xname, class_names=yname, filled=True, fontsize=9) 
plt.show()